## Mono FM Demodulator

In [ ]:
import numpy as np
from scipy.signal import hilbert

### Read from the recorded data from RTL-SDR

In [3]:
with open('fm941.bin', mode='rb') as file: # b is important -> binary
    x = file.read()

y = [(float(i)-127.5) for i in x]
y_re = y[0::2]
y_im = y[1::2]
len(y_re)

10000000

### Step 1: Convert y_re and y_im into a Complex Signal
First, combine y_re (real part) and y_im (imaginary part) to form a single complex array. This complex array represents the I/Q data from your RTL-SDR.

In [4]:
import numpy as np

# Combine the real and imaginary parts to form the complex baseband signal
y_complex = np.array(y_re) + 1j*np.array(y_im)

### Step 2: Define the FM Demodulation Function
To demodulate the FM signal, you can use the phase derivative method. This method involves calculating the instantaneous phase of the signal and then finding its derivative, which is proportional to the original modulating signal.

In [5]:
def fm_demodulate(complex_signal, fm_deviation):
    """
    Demodulate a frequency modulated signal using the phase derivative method.
    
    Args:
    complex_signal (numpy.ndarray): The complex I/Q data samples.
    fm_deviation (float): The peak frequency deviation of the FM signal in Hz.
    
    Returns:
    numpy.ndarray: The demodulated signal.
    """
    # Calculate the instantaneous phase of the complex signal
    instantaneous_phase = np.unwrap(np.angle(complex_signal))
    
    # Calculate the derivative of the phase
    instantaneous_freq = np.diff(instantaneous_phase)
    
    # Convert the derivative of the phase to a demodulated signal
    # The factor (1/(2*pi)) converts from radians per sample to cycles per sample
    # Multiply by sample_rate to convert to cycles per second (Hz)
    demodulated_signal = (instantaneous_freq / (2 * np.pi)) * sample_rate / fm_deviation
    
    return demodulated_signal


### Step 3: Specify the FM Radio Parameters
Set the sample rate and FM deviation. These values depend on how you configured the RTL-SDR when you captured the data. For typical wideband FM broadcasts, the frequency deviation is around 75 kHz.

In [15]:
sample_rate = 2.4e6  # Example: 2.4 MHz sample rate
fm_deviation = 75e3  # Example: 75 kHz FM deviation


### Step 4: Demodulate the Signal
Call the fm_demodulate function with your complex signal.

In [16]:
# Demodulate the FM signal
demodulated_audio = fm_demodulate(y_complex, fm_deviation)

# Since np.diff reduces the length of the array by 1, optionally you can append a zero
demodulated_audio = np.append(demodulated_audio, 0)  # Maintain original length for consistency


### Step 5: Optionally Save or Play the Demodulated Audio
You might want to listen to the demodulated signal or save it to a file. Here's how you could save it using scipy.io.wavfile:

In [17]:
from scipy.io.wavfile import write

# Normalize and scale to int16 for WAV file
audio_scaled = np.int16((demodulated_audio / np.max(np.abs(demodulated_audio))) * 32767)

# Write to a WAV file
write('demodulated_audio.wav', int(sample_rate), audio_scaled)


#### This code sequence provides a straightforward method for FM demodulation using Python, applicable for processing RTL-SDR recorded signals in scenarios such as amateur radio, FM broadcast reception, and other similar applications. Adjust the sample_rate and fm_deviation according to the specific details of your signal capture setup.